## Importing of Libraries


In [1]:
import time
from copy import deepcopy

import numpy as np
import torch
from dlordinal.datasets import FGNet
from dlordinal.losses import (
    BetaCrossEntropyLoss,
    BinomialCrossEntropyLoss,
    ExponentialRegularisedCrossEntropyLoss,
    PoissonCrossEntropyLoss,
    TriangularCrossEntropyLoss,
    GeneralTriangularCrossEntropyLoss,
    WKLoss,
)
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    mean_absolute_error,
)
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from torch import cuda, nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor
from tqdm import tqdm

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the *DataLoader*, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [2]:
optimiser_params = {
    'lr': 1e-3,
    'bs': 200,
    'epochs': 5,
    's': 2,
    'c': 0.2,
    'beta': 0.5
}

workers = 3

Now we use the *FGNet* method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the *StratifiedShuffleSplit* method. Finally, with all the partitions, we load the images using a method called *DataLoader*.

In [14]:
fgnet = FGNet(root="./datasets/fgnet", download=True, process_data=True)

complete_train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

num_classes = len(complete_train_data.classes)
classes = complete_train_data.classes
targets = complete_train_data.targets

# Create a validation split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
sss_splits = list(
    sss.split(X=np.zeros(len(complete_train_data)), y=complete_train_data.targets)
)
train_idx, val_idx = sss_splits[0]

# Create subsets for training and validation
train_data = Subset(complete_train_data, train_idx)
val_data = Subset(complete_train_data, val_idx)

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Create dataloaders
train_dataloader = DataLoader(
    train_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
val_dataloader = DataLoader(
    val_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
test_dataloader = DataLoader(
    test_data, batch_size=optimiser_params["bs"], shuffle=False, num_workers=workers
)

# Get image shape
img_shape = None
for X, _ in train_dataloader:
    img_shape = list(X.shape[1:])
    break
print(f"Detected image shape: {img_shape}")

# Define class weights for imbalanced datasets
class_weights = class_weight.compute_class_weight(
    "balanced", classes=[int(c) for c in classes], y=targets
)
print(f"{class_weights=}")
class_weights = (
    torch.from_numpy(class_weights).float().to(device)
)  # Transform to Tensor

Files already downloaded and verified
Files already processed and verified
Files already split and verified
num_classes=6
Using cuda device


Detected image shape: [3, 128, 128]
class_weights=array([1.60843373, 0.55394191, 1.02692308, 0.78070175, 1.12184874,
       2.34210526])


## Model

We are using a pretrained *ResNet* model, which has previously been trained on ImageNet. We are modifying the last fully connected layer to match the number of classes in our dataset.

Finally, we define the *Adam* optimiser, which is used to adjust the network's weights and minimize the error of a loss function.

In [4]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Optimizer and scheduler
optimizer = Adam(model.parameters(), lr=optimiser_params['lr'])

## Loss Function

Is a regularized loss which uses soft labels obtained from binomial distributions [1].

The binomial distribution is the following:

$$
P_j(q) = \binom{Q}{q} p_q^j (1 - p_q)^{Q-q}
$$

where $p_q$ is the parameter associated with the distribution of the class $C_q$, and $Q$ is the number of classes.

There are other loss functions implemented in the package such as:
- Beta Cross Entropy Loss [2]
- Exponential Regularised Cross Entropy Loss [1]
- Poisson Cross Entropy Loss [3]
- Weighted Kappa Loss [4]
- Triangular Cross Entropy Loss [5]
- Generalised Triangular Cross Entropy Loss [6]

[1]: Vargas, Víctor Manuel, et al. (2023). *Exponential loss regularisation for encouraging ordinal constraint to shotgun stocks quality assessment.* Applied Soft Computing, 138, 110191. doi:10.1016/j.asoc.2023.110191

[2]: Vargas, Víctor Manuel et al. (2022). *Unimodal regularisation based on beta distribution for deep ordinal regression.* Pattern Recognition, 122, 108310. Elsevier. doi.org/10.1016/j.patcog.2021.108310

[3]: Liu, Xiaofeng et al. (2020). *Unimodal regularized neuron stick-breaking for ordinal classification.* Neurocomputing, 388, 34-44. doi.org/10.1016/j.neucom.2020.01.025

[4]: de La Torre, J., Puig, D., & Valls, A. (2018). *Weighted kappa loss function for multi-class classification of ordinal data in deep learning.* Pattern Recognition Letters, 105, 144-154. doi.org/10.1016/j.patrec.2017.05.018

[5]: Víctor Manuel Vargas, Pedro Antonio Gutiérrez, Javier Barbero-Gómez, and César Hervás-Martínez (2023). *Soft Labelling Based on Triangular Distributions for Ordinal Classification.* Information Fusion, 93, 258--267. doi.org/10.1016/j.inffus.2023.01.003

[6]: Víctor Manuel Vargas, Antonio Manuel Durán-Rosal, David Guijo-Rubio, Pedro Antonio Gutiérrez-Peña, and César Hervás-Martínez (2023). *Generalised Triangular Distributions for ordinal deep learning: novel proposal and optimisation.* Information Sciences, 648, 1--17. doi.org/10.1016/j.ins.2023.119606

In [13]:
loss_fn = BinomialCrossEntropyLoss(num_classes=num_classes).to(device)

## Metrics

Here, we provide code for training and testing the ResNet model, along with various metrics that demonstrate its performance.

In [6]:
# Metrics computation
def compute_metrics(y_true: np.ndarray, 
    y_pred: np.ndarray, 
    num_classes: int):

    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)

    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    labels = range(0, num_classes)

    # Metrics calculation
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic', labels=labels)
    ms = minimum_sensitivity(y_true, y_pred, labels=labels)
    mae = mean_absolute_error(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    off1 = accuracy_off1(y_true, y_pred, labels=labels)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

    metrics = {
        'QWK': qwk,
        'MS': ms,
        'MAE': mae,
        'CCR': acc,
        '1-off': off1,
        'Confusion matrix': conf_mat
    }

    return metrics

def _compute_sensitivities(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

	sum = np.sum(conf_mat, axis=1)
	mask = np.eye(conf_mat.shape[0], conf_mat.shape[1])
	correct = np.sum(conf_mat * mask, axis=1)
	sensitivities = correct / sum

	sensitivities = sensitivities[~np.isnan(sensitivities)]

	return sensitivities

def minimum_sensitivity(y_true, y_pred, labels=None):
	return np.min(_compute_sensitivities(y_true, y_pred, labels=labels))

def accuracy_off1(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
	n = conf_mat.shape[0]
	mask = np.eye(n, n) + np.eye(n, n, k=1), + np.eye(n, n, k=-1)
	correct = mask * conf_mat

	return 1.0 * np.sum(correct) / np.sum(conf_mat)

def print_metrics(metrics):
    print("")
    print('Confusion matrix :\n{}'.format(metrics['Confusion matrix']))
    print("")
    print('MS: {:.4f}'.format(metrics['MS']))
    print("")
    print('QWK: {:.4f}'.format(metrics['QWK']))
    print("")
    print('MAE: {:.4f}'.format(metrics['MAE']))
    print("")
    print('CCR: {:.4f}'.format(metrics['CCR']))
    print("")
    print('1-off: {:.4f}'.format(metrics['1-off']))

## Training Process

In [11]:
def train(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    H: dict,
    num_classes: int,
):  # H: dict
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Train progress")
    model.train()
    mean_loss, accuracy = 0, 0
    y_pred, y_true = None, None

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Inputs and labels to device

        # Compute prediction error and accuracy of the training process
        pred = model(X)
        loss = loss_fn(pred, y)

        mean_loss += loss
        accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Stack predictions and true labels to determine the confusion matrix
        pred_np = pred.argmax(1).cpu().detach().numpy()
        true_np = y.cpu().detach().numpy()
        if y_pred is None:
            y_pred = pred_np
        else:
            y_pred = np.concatenate((y_pred, pred_np))

        if y_true is None:
            y_true = true_np
        else:
            y_true = np.concatenate((y_true, true_np))

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy)
        progress_bar.update(1)

    accuracy /= size
    mean_loss /= num_batches

    H["train_loss"].append(loss.cpu().detach().numpy())
    H["train_acc"].append(accuracy)

    # Confusion matrix for training
    labels = range(0, num_classes)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    print("")
    print("Train Confusion matrix :\n{}".format(conf_mat))
    print("")

    return accuracy, mean_loss

## Test Process

In [9]:
def test(
    test_dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: torch.device,
    num_classes: int,
):
    num_batches = len(test_dataloader)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Test progress")
    model.eval()
    test_loss = 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)  # inputs and labels to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # Stack predictions and true labels
            pred_np = pred.argmax(1).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

            # Update progress bar
            progress_bar.set_postfix(loss=test_loss / (batch + 1))
            progress_bar.update(1)

    test_loss /= num_batches
    metrics = compute_metrics(y_true, y_pred, num_classes)
    print_metrics(metrics)

    return metrics, test_loss

## Validation Process

In [10]:
def validate(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: torch.device,
    H: dict,
    num_classes: int,
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, accuracy = 0, 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss += loss_fn(pred, y)
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

            pred_np = pred.argmax(1).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

    accuracy /= size
    loss /= num_batches

    H["val_loss"].append(loss.cpu().detach().numpy())
    H["val_acc"].append(accuracy)

    metrics = compute_metrics(y_true, y_pred, num_classes)

    return metrics, accuracy, loss

## Results

In [12]:
H = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# To store validation metrics
validation_metrics = {}

# Definition to store best model weights
best_model_weights = model.state_dict()
best_qwk = 0.0

# Start time
start_time = time.time()

for e in range(optimiser_params["epochs"]):
    train_acc, train_loss = train(
        train_dataloader, model, loss_fn, optimizer, device, H, num_classes=num_classes
    )
    validation_metrics, val_acc, val_loss = validate(
        val_dataloader, model, loss_fn, device, H, num_classes=num_classes
    )

    if validation_metrics["QWK"] >= best_qwk:
        best_qwk = validation_metrics["QWK"]
        best_model_weights = deepcopy(model.state_dict())

    print("[INFO] EPOCH: {}/{}".format(e + 1, optimiser_params["epochs"]))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(train_loss, train_acc))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(val_loss, val_acc))

# Store last train error
train_error = H["train_loss"][-1]

# Restore best weights
model.load_state_dict(best_model_weights)

# Start evaluation
print("[INFO] Network evaluation ...")

test_metrics, test_loss = test(
    test_dataloader, model, loss_fn, device, num_classes=num_classes
)

# End time
end_time = time.time()
print("\n[INFO] Total training time: {:.2f}s".format(end_time - start_time))

Train progress:  50%|████████████            | 2/4 [00:01<00:01,  1.69it/s, accuracy=106, loss=1.62]

tensor([[ 0.4965,  0.5200,  0.2156,  0.9261, -0.6116,  1.0949],
        [-0.4715, -0.7595,  1.1330,  0.7932,  0.0749,  1.2884],
        [ 0.8929,  0.5330,  0.0984,  0.3900, -0.7238,  0.4939],
        ...,
        [ 0.3272, -0.5772, -0.2451, -0.1964, -1.1121,  1.1002],
        [ 0.3922, -1.2060, -0.2031,  0.7491, -0.4196, -0.4266],
        [-0.1965, -0.1802,  1.1205,  0.7332,  0.4739,  0.6164]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([5, 3, 1, 3, 1, 5, 4, 1, 3, 3, 4, 1, 1, 1, 5, 1, 1, 0, 1, 4, 5, 1, 1, 0,
        3, 2, 5, 3, 2, 1, 3, 2, 3, 1, 0, 2, 4, 0, 3, 4, 5, 3, 1, 5, 1, 4, 1, 1,
        2, 3, 5, 1, 3, 3, 0, 2, 2, 1, 4, 1, 1, 2, 2, 4, 2, 3, 1, 3, 3, 1, 2, 1,
        4, 3, 1, 3, 4, 1, 0, 5, 1, 2, 3, 0, 4, 2, 2, 2, 1, 1, 4, 4, 1, 2, 0, 4,
        1, 2, 1, 1, 1, 4, 4, 3, 2, 2, 0, 0, 4, 3, 5, 0, 2, 4, 3, 5, 1, 2, 0, 2,
        1, 4, 2, 4, 3, 3, 1, 4, 1, 1, 3, 5, 4, 4, 3, 3, 4, 3, 2, 4, 1, 1, 3, 2,
        3, 5, 1, 2, 3, 2, 1, 0, 5, 1, 4, 4, 4, 4, 1, 3, 3, 1, 3, 2, 0, 5

Train progress: 100%|████████████████████████| 4/4 [00:01<00:00,  2.47it/s, accuracy=232, loss=1.36]

tensor([[ 1.6175,  1.8317,  0.9192, -1.5188, -1.9413, -1.6520],
        [-1.1876,  1.4564, -0.1189,  0.0677, -0.0636,  0.0154],
        [ 0.4680,  2.9719,  0.8327, -0.0130, -2.0878, -2.2973],
        ...,
        [ 4.6847,  1.5378,  0.0068, -1.4120, -0.5424, -0.9518],
        [ 0.8759,  0.8369, -0.2155, -0.0173,  0.9739, -1.0331],
        [ 5.9574,  1.9509, -0.8973, -4.1554, -0.7843, -2.1261]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([1, 4, 3, 3, 3, 2, 3, 1, 2, 3, 3, 4, 1, 2, 2, 4, 4, 1, 2, 2, 2, 1, 1, 3,
        3, 5, 5, 4, 5, 2, 0, 3, 1, 1, 3, 4, 3, 2, 1, 1, 1, 1, 3, 1, 3, 5, 1, 4,
        1, 3, 3, 1, 1, 2, 1, 2, 3, 4, 2, 2, 0, 0, 1, 3, 4, 4, 1, 1, 3, 4, 3, 3,
        3, 0, 3, 1, 3, 0, 0, 1, 5, 1, 3, 1, 1, 2, 5, 4, 5, 3, 4, 0, 1, 2, 0, 3,
        0, 1, 1, 3, 1, 2, 2, 0, 5, 1, 1, 3, 0, 3, 4, 2, 2, 4, 1, 0, 3, 2, 1, 0,
        1, 4, 4, 1, 3, 5, 0, 3, 2, 1, 3, 3, 3, 1, 4, 1, 3, 4, 0, 1, 3, 1, 2, 1,
        1, 1, 3, 2, 4, 4, 1, 3, 3, 3, 5, 0, 2, 4, 1, 3, 3, 1, 4, 2, 1, 3

[INFO] EPOCH: 1/5
Train loss: 1.656975, Train accuracy: 0.3412
Val loss: 2.432539, Val accuracy: 0.2562



...

Train progress: 100%|███████████████████████| 4/4 [00:00<00:00,  4.81it/s, accuracy=633, loss=0.888]

tensor([[ -3.2497,   0.5673,   3.1601,   2.3869,  -0.0335,  -2.0497],
        [ -2.4593,   0.5386,   1.7118,   2.2894,   0.1709,  -2.1822],
        [ -1.2751,   1.5196,   3.4448,   2.3384,  -1.4411,  -4.2802],
        [  2.0649,   3.7405,   2.5357,  -1.3918,  -2.4507,  -4.4183],
        [ -4.1344,  -1.1459,   0.1346,   2.2316,   2.6874,   1.2632],
        [ -8.3915,  -3.0208,   0.3923,   3.8464,   6.2180,   4.2387],
        [  3.3730,   4.8781,   2.5860,  -2.2050,  -3.4234,  -5.5517],
        [  8.3960,   4.3488,   2.7891,  -4.0404,  -4.0036,  -6.3879],
        [-10.3759,  -3.5367,  -1.6626,   3.1039,   7.0578,   6.8458],
        [  1.1714,   2.6555,   1.2416,  -1.1252,  -2.0335,  -3.0712],
        [ -6.8211,  -2.5425,   0.7500,   3.5888,   4.6562,   3.0443],
        [ -7.5116,  -1.6554,  -0.4702,   2.0625,   4.2912,   4.5781],
        [  3.4008,   4.7224,   3.2029,  -1.3657,  -4.1060,  -5.6949],
        [ -0.3255,   2.2532,   3.4362,   1.6081,  -2.2351,  -4.9790],
        [ -4.2005,  

[INFO] EPOCH: 5/5
Train loss: 0.888149, Train accuracy: 0.9309
Val loss: 1.563793, Val accuracy: 0.5372

[INFO] Network evaluation ...


Test progress: 100%|███████████████████████████████████████| 2/2 [00:01<00:00,  1.21it/s, loss=4.43]


Confusion matrix :
[[17  8  0  0  1  0]
 [ 8 40 10  2  1  0]
 [ 0  8  7 18  1  0]
 [ 0  2  8 20  9  0]
 [ 0  2  2  9 16  0]
 [ 0  1  0  4  7  0]]

MS: 0.0000

QWK: 0.7589

MAE: 0.6169

CCR: 0.4975

1-off: 0.9204

[INFO] Total training time: 8.96s
